# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [22]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,4.65,91,100,12.09,TF,1701489168
1,1,ilulissat,69.2167,-51.1000,0.01,40,22,9.77,GL,1701489194
2,2,kattivakkam,13.2167,80.3167,26.71,91,99,9.19,IN,1701489194
3,3,el bolson,-41.9667,-71.5167,1.66,98,39,2.31,AR,1701489194
4,4,afaahiti,-17.7500,-149.2833,28.93,80,41,7.03,PF,1701489195


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size="Humidity",
    color="City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_filter_df = city_data_df.loc[(city_data_df["Max Temp"]<27) & (city_data_df["Max Temp"]>25) &
                                      (city_data_df["Cloudiness"]>50) & (city_data_df["Wind Speed"]<4.5)&
                                      (city_data_df["Humidity"]>75)]

# Drop any rows with null values
city_data_filter_df.dropna()

# Display sample data
city_data_filter_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,43,san antonio de pale,-1.4014,5.6325,26.34,82,100,2.47,GQ,1701489201
76,76,hawaiian paradise park,19.5933,-154.9731,25.73,90,75,2.57,US,1701489206
137,137,saint-philippe,-21.3585,55.7679,25.77,76,100,3.15,RE,1701489216
156,156,acarau,-2.8856,-40.1200,26.02,82,59,4.04,BR,1701489218
162,162,luis correia,-2.8792,-41.6669,26.69,83,100,4.46,BR,1701489219
167,167,haiku-pauwela,20.9219,-156.3051,25.58,85,75,2.57,US,1701489220
193,193,sao tome,0.3365,6.7273,25.11,87,100,2.01,ST,1701489223
231,231,ciudad lazaro cardenas,17.9583,-102.2000,25.19,77,100,1.66,MX,1701489227
232,232,freetown,8.4840,-13.2299,26.39,85,100,1.36,SL,1701489227
271,271,brisas de zicatela,15.8369,-97.0419,26.10,78,70,1.77,MX,1701489234


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_filter_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
43,san antonio de pale,GQ,-1.4014,5.6325,82,
76,hawaiian paradise park,US,19.5933,-154.9731,90,
137,saint-philippe,RE,-21.3585,55.7679,76,
156,acarau,BR,-2.8856,-40.1200,82,
162,luis correia,BR,-2.8792,-41.6669,83,
167,haiku-pauwela,US,20.9219,-156.3051,85,
193,sao tome,ST,0.3365,6.7273,87,
231,ciudad lazaro cardenas,MX,17.9583,-102.2000,77,
232,freetown,SL,8.4840,-13.2299,85,
271,brisas de zicatela,MX,15.8369,-97.0419,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":1,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{row["Lng"]},{row["Lat"]},{radius}'
    params["bias"] = f'proximity:{row["Lng"]},{row["Lat"]}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
san antonio de pale - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
saint-philippe - nearest hotel: Le Baril
acarau - nearest hotel: No hotel found
luis correia - nearest hotel: Hotel Amaro
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
sao tome - nearest hotel: Hotel Baia
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
freetown - nearest hotel: Formal Leona hotel
brisas de zicatela - nearest hotel: Casa de Olas
cumbum - nearest hotel: No hotel found
imbituba - nearest hotel: Silvestre Praia Hotel
hilo - nearest hotel: Dolphin Bay Hotel
bu'aale - nearest hotel: No hotel found
porto novo - nearest hotel: Hôtel 6500 CFA
jamaame - nearest hotel: No hotel found
bonthe - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
43,san antonio de pale,GQ,-1.4014,5.6325,82,No hotel found
76,hawaiian paradise park,US,19.5933,-154.9731,90,No hotel found
137,saint-philippe,RE,-21.3585,55.7679,76,Le Baril
156,acarau,BR,-2.8856,-40.1200,82,No hotel found
162,luis correia,BR,-2.8792,-41.6669,83,Hotel Amaro
167,haiku-pauwela,US,20.9219,-156.3051,85,Inn At Mama's Fish House
193,sao tome,ST,0.3365,6.7273,87,Hotel Baia
231,ciudad lazaro cardenas,MX,17.9583,-102.2000,77,Hotel Sol del Pacífico
232,freetown,SL,8.4840,-13.2299,85,Formal Leona hotel
271,brisas de zicatela,MX,15.8369,-97.0419,78,Casa de Olas


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [27]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size="Humidity",
    color="City",
    hover_cols=["Hotel Name","Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)